# Using Neo4j LlamaIndex query engine with AG2 agents for GraphRAG Question & Answering

 AG2 provides GraphRAG integration using agent capabilities. This is an example to integrate Neo4j  property graph and LlamaIndex graph query engine.

````{=mdx}
:::info Requirements
llama-index dependencies, which is required to use Neo4j prpoerty graph

```bash
pip install llama-index==0.11.8 llama-index-graph-stores-neo4j==0.3.0 llama-index-core==0.11.8
```


## Set Configuration and OpenAI API Key

By default, in order to use FalkorDB you need to have an OpenAI key in your environment variable `OPENAI_API_KEY`.

You can utilise an OAI_CONFIG_LIST file and extract the OpenAI API key and put it in the environment, as will be shown in the following cell.

Alternatively, you can load the environment variable yourself.

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

In [1]:
import os

import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST", file_location="../")

# Put the OpenAI API key into the environment
os.environ["OPENAI_API_KEY"] = config_list[0]["api_key"]

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# This is needed to allow nested asyncio calls for Neo4j in Jupyter
import nest_asyncio

nest_asyncio.apply()

## Key Information: Using Neo4j with LLM Models 🚀

> **Important**  
> - **Default Models**:
>   - **Question Answering**: OpenAI's `GPT-3.5-turbo` with `temperature=0.0`.
>   - **Embedding**: OpenAI's `text-embedding-3-small`.
> 
> - **Customization**:
>   You can change these defaults by setting the following parameters on the `Neo4jGraphQueryEngine`:
>   - `llm`: Specify a LLM instance with a llm you like.
>   - `embedding`: Specify a BaseEmbedding instance with a embedding model.
> - **Refer to**
>   - https://docs.llamaindex.ai/en/stable/module_guides/models/llms/
>   - https://docs.llamaindex.ai/en/stable/examples/property_graph/graph_store/


## Create a Knowledge Graph with Your Own Data

**Note:** You need to have a Neo4j database running. If you are running one in a Docker container, please ensure your Docker network is setup to allow access to it. 

In this example, the Neo4j endpoint is set to host="bolt://172.17.0.4" and port=7687, please adjust accordingly. For how to spin up a Neo4j with Docker, you can refer to [this](https://docs.llamaindex.ai/en/stable/examples/property_graph/property_graph_neo4j/#:~:text=stores%2Dneo4j-,Docker%20Setup,%C2%B6,-To%20launch%20Neo4j)

Below, we have some sample data from Paul Grahma's [essay](https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt).

We then initialise the database with that text document, creating the graph in Neo4j.

### A Simple Example

In this example, the graph schema is auto-generated. This allows you to load data without specifying the specific types of entities and relationships that will make up the database (however, this may not be optimal and not cost efficient). 
First, we create a Neo4j property graph (knowledge graph) with Paul Grahma's essay.

In [3]:
# load documents
from autogen.agentchat.contrib.graph_rag.document import Document, DocumentType

input_path = "../test/agentchat/contrib/graph_rag/paul_graham_essay.txt"
input_documents = [Document(doctype=DocumentType.TEXT, path_or_url=input_path)]

In [4]:
# Auto generate graph schema from unstructured data
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.graph_rag.neo4j_graph_query_engine import Neo4jGraphQueryEngine

# Create FalkorGraphQueryEngine
query_engine = Neo4jGraphQueryEngine(
    username="neo4j",  # Change if you reset username
    password="password",  # Change if you reset password
    host="bolt://172.17.0.3",  # Change
    port=7687,  # if needed
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.0),  # Default, no need to specify
    embedding=OpenAIEmbedding(model_name="text-embedding-3-small"),  # except you want to use a different model
    database="neo4j",  # Change if you want to store the graphh in your custom database
)

# Ingest data and initialize the database
query_engine.init_db(input_doc=input_documents)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"
Generating embeddings: 100%|██████████| 2/2 [00:04<00:00,  2.15s/it]
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (e, row) { ... }} {position: line

### Add capability to a ConversableAgent and query them

In [4]:
from autogen.agentchat.contrib.graph_rag.neo4j_graph_rag_capability import Neo4jGraphCapability

# Create a ConversableAgent (no LLM configuration)
graph_rag_agent = ConversableAgent(
    name="paul_graham_agent",
    human_input_mode="NEVER",
)

# Associate the capability with the agent
graph_rag_capability = Neo4jGraphCapability(query_engine)
graph_rag_capability.add_to_agent(graph_rag_agent)

# Create a user proxy agent to converse with our RAG agent
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
)

user_proxy.initiate_chat(graph_rag_agent, message="What happened at Interleaf and Viaweb?")

user_proxy (to paul_graham_agent):

What happened at Interleaf and Viaweb?

--------------------------------------------------------------------------------
paul_graham_agent (to user_proxy):

Interleaf was a company that had smart people and built impressive technology but got crushed by Moore's Law in the 1990s. On the other hand, Viaweb was a company founded by the author and his partner to put art galleries online. However, they realized that art galleries didn't want to be online, and they pivoted to building online stores instead. They developed software to generate web stores and transitioned to creating web apps that allowed users to control the software through a browser, leading to the establishment of Viaweb as a pioneering web application company.

--------------------------------------------------------------------------------
user_proxy (to paul_graham_agent):

What did Paul Graham do at Interleaf

--------------------------------------------------------------------------

ChatResult(chat_id=None, chat_history=[{'content': 'What happened at Interleaf and Viaweb?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': "Interleaf was a company that had smart people and built impressive technology but got crushed by Moore's Law in the 1990s. On the other hand, Viaweb was a company founded by the author and his partner to put art galleries online. However, they realized that art galleries didn't want to be online, and they pivoted to building online stores instead. They developed software to generate web stores and transitioned to creating web apps that allowed users to control the software through a browser, leading to the establishment of Viaweb as a pioneering web application company.", 'role': 'user', 'name': 'paul_graham_agent'}, {'content': 'What did Paul Graham do at Interleaf', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'Paul Graham did freelance Lisp hacking work at Interleaf.', 'role': 'user', 'name': 'paul_graham_agent'}, {'content'

### Revisit the example by defining custom entities, relations and schema

In [9]:
from typing import Literal

# best practice to use upper-case
entities = Literal["PERSON", "PLACE", "ORGANIZATION"]  #
relations = Literal["HAS", "PART_OF", "WORKED_ON", "WORKED_WITH", "WORKED_AT"]

# define which entities can have which relations
validation_schema = {
    "PERSON": ["HAS", "PART_OF", "WORKED_ON", "WORKED_WITH", "WORKED_AT"],
    "PLACE": ["HAS", "PART_OF", "WORKED_AT"],
    "ORGANIZATION": ["HAS", "PART_OF", "WORKED_WITH"],
}

# Create FalkorGraphQueryEngine
query_engine = Neo4jGraphQueryEngine(
    username="neo4j",  # Change if you reset username
    password="password",  # Change if you reset password
    host="bolt://172.17.0.3",  # Change
    port=7687,  # if needed
    database="neo4j",  # Change if you want to store the graphh in your custom database
    entities=entities,  # possible entities
    relations=relations,  # possible relations
    validation_schema=validation_schema,  # schema to validate the extracted triplets
    strict=True,  # enofrce the extracted triplets to be in the schema
)

# Ingest data and initialize the database
query_engine.init_db(input_doc=input_documents)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"
Generating embeddings: 100%|██████████| 4/4 [00:01<00:00,  3.92it/s]
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (e, row) { ... }} {position: line

### Add capability to a ConversableAgent and query them again
You should find the answers conform to your custom schema 

In [12]:
from autogen.agentchat.contrib.graph_rag.neo4j_graph_rag_capability import Neo4jGraphCapability

# Create a ConversableAgent (no LLM configuration)
graph_rag_agent = ConversableAgent(
    name="paul_graham_agent",
    human_input_mode="NEVER",
)

# Associate the capability with the agent
graph_rag_capability = Neo4jGraphCapability(query_engine)
graph_rag_capability.add_to_agent(graph_rag_agent)

# Create a user proxy agent to converse with our RAG agent
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
)

user_proxy.initiate_chat(graph_rag_agent, message="Which companies did Paul Graham work for?")

user_proxy (to paul_graham_agent):

Which companies did Paul Graham work for?

--------------------------------------------------------------------------------
paul_graham_agent (to user_proxy):

Paul Graham worked for Y Combinator (YC).

--------------------------------------------------------------------------------
user_proxy (to paul_graham_agent):

who did he worked with?

--------------------------------------------------------------------------------
paul_graham_agent (to user_proxy):

Jessica

--------------------------------------------------------------------------------
user_proxy (to paul_graham_agent):

Give me more people he worked with

--------------------------------------------------------------------------------
paul_graham_agent (to user_proxy):

Trevor Blackwell, John Collison, Patrick Collison, Daniel Gackle, Ralph Hazell, Robert Morris, and Harj Taggar.

--------------------------------------------------------------------------------
user_proxy (to paul_graham_ag

ChatResult(chat_id=None, chat_history=[{'content': 'Which companies did Paul Graham work for?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'Paul Graham worked for Y Combinator (YC).', 'role': 'user', 'name': 'paul_graham_agent'}, {'content': 'who did he worked with?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'Jessica', 'role': 'user', 'name': 'paul_graham_agent'}, {'content': 'Give me more people he worked with', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'Trevor Blackwell, John Collison, Patrick Collison, Daniel Gackle, Ralph Hazell, Robert Morris, and Harj Taggar.', 'role': 'user', 'name': 'paul_graham_agent'}, {'content': 'Did he worked with Joe Biden?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'No, there is no mention or indication in the provided context information that he worked with Joe Biden.', 'role': 'user', 'name': 'paul_graham_agent'}], summary='No, there is no mention or indication in the provided context information

### You can add new documents to the existing knoweldge graph!

In [5]:
input_path = "../test/agentchat/contrib/graph_rag/the_matrix.txt"
input_documents = [Document(doctype=DocumentType.TEXT, path_or_url=input_path)]

_ = query_engine.add_records(input_documents)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.45it/s]
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (e, row) { ... }} {position: line: 10, column: 21, offset: 397} for query: "\n                    UNWIND $data AS row\n                    MERGE (e:__Node__ {id: row.id})\n                    SET e += apoc.map.clean(row.properties, [], [])\n                    SET e.name = row.name, e:`__Entity__`\n                    WITH e, row\n                    CALL apoc.create.addLabels(e, [row.label])\n                    YIELD node\n                    WITH e, row\n                    CALL {\n                        WITH e, row\n                        WITH e, row\n                        WHERE row.embedding IS NOT N

### Now let's create a new graph rag agent and some quetions related to both 2 documents

In [7]:
from autogen.agentchat.contrib.graph_rag.neo4j_graph_rag_capability import Neo4jGraphCapability

# Create a ConversableAgent (no LLM configuration)
graph_rag_agent = ConversableAgent(
    name="paul_graham_agent",
    human_input_mode="NEVER",
)

# Associate the capability with the agent
graph_rag_capability = Neo4jGraphCapability(query_engine)
graph_rag_capability.add_to_agent(graph_rag_agent)

# Create a user proxy agent to converse with our RAG agent
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
)

user_proxy.initiate_chat(graph_rag_agent, message="Who acted at 'The Matrix'?")

user_proxy (to paul_graham_agent):

Who acted at 'The Matrix'?

--------------------------------------------------------------------------------
paul_graham_agent (to user_proxy):

Keanu Reeves, Carrie-Anne Moss, Laurence Fishburne, and Anthony Ray Parker acted in 'The Matrix'.

--------------------------------------------------------------------------------
user_proxy (to paul_graham_agent):

What did Paul Graham do at Interleaf?

--------------------------------------------------------------------------------
paul_graham_agent (to user_proxy):

Paul Graham was involved with Interleaf, a company that had smart people and built impressive technology, but ultimately got crushed by Moore's Law.

--------------------------------------------------------------------------------
user_proxy (to paul_graham_agent):

Did Paul Graham acted in 'The Matrix'?

--------------------------------------------------------------------------------
paul_graham_agent (to user_proxy):

No, there is no informa

ChatResult(chat_id=None, chat_history=[{'content': "Who acted at 'The Matrix'?", 'role': 'assistant', 'name': 'user_proxy'}, {'content': "Keanu Reeves, Carrie-Anne Moss, Laurence Fishburne, and Anthony Ray Parker acted in 'The Matrix'.", 'role': 'user', 'name': 'paul_graham_agent'}, {'content': 'What did Paul Graham do at Interleaf?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': "Paul Graham was involved with Interleaf, a company that had smart people and built impressive technology, but ultimately got crushed by Moore's Law.", 'role': 'user', 'name': 'paul_graham_agent'}, {'content': "Did Paul Graham acted in 'The Matrix'?", 'role': 'assistant', 'name': 'user_proxy'}, {'content': "No, there is no information provided in the context about Paul Graham acting in 'The Matrix'.", 'role': 'user', 'name': 'paul_graham_agent'}, {'content': "Did Paul Graham do anything for 'The Matrix'?", 'role': 'assistant', 'name': 'user_proxy'}, {'content': "Paul Graham did not do anything for 'T